# MGMT radiomics baseline pipeline (dummy → RF → GA-RF)

This notebook shows how to:
- load the four pre-split CSV files you created,
- get floor performance using dummy classifiers,
- train a plain Random Forest on all radiomics features,
- plug in a GA-selected feature subset to train GA-RF.

All comments in the code are in English so you can paste this into GitHub or a report.


In [37]:
import os
import random
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score

## 1. Load data
We load `X_train.csv`, `X_val.csv`, `y_train.csv`, and `y_val.csv`.  
Labels are squeezed to 1D so that scikit-learn can use them directly.  
If your labels are strings like `"Methylated"`, you can map them to integers right after loading.


In [38]:
# set seeds for reproducibility
random.seed(42)
np.random.seed(42)

# read feature matrices
X_train = pd.read_csv("../data/X_train.csv")   # training features
X_val   = pd.read_csv("../data/X_val.csv")     # validation features

# read target vectors (squeeze → Series, not DataFrame)
y_train = pd.read_csv("../data/y_train.csv").squeeze()
y_val   = pd.read_csv("../data/y_val.csv").squeeze()

# quick sanity check on shapes
print("X_train:", X_train.shape)   # (n_train, d)
print("X_val:  ", X_val.shape)     # (n_val, d)
print("y_train:", y_train.shape)   # (n_train,)
print("y_val:  ", y_val.shape)     # (n_val,)


# --- GA needs these two globals ---
New_FS = X_train.copy()              # GA will select columns from here
y_trn  = y_train.reset_index(drop=True)

print("X_train:", X_train.shape)
print("X_val:  ", X_val.shape)
print("y_train:", y_train.shape)
print("y_val:  ", y_val.shape)

GA_POP_PATH   = "../data/ga_pop.npy"
GA_SCORE_PATH = "../data/ga_scores.npy"

size  = 50
n_feat = New_FS.shape[1]   # now this works

# if labels are text, map to 0/1 here
# y_train = (y_train == "Methylated").astype(int)
# y_val   = (y_val == "Methylated").astype(int)


X_train: (42, 725)
X_val:   (11, 725)
y_train: (42,)
y_val:   (11,)
X_train: (42, 725)
X_val:   (11, 725)
y_train: (42,)
y_val:   (11,)


## 2. Dummy baselines
We train two dummy models to see the minimum performance on this split.  
`most_frequent` = always predict the majority class.  
`stratified` = predict classes according to training distribution (a bit harder baseline).


In [39]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# dummy that always picks the most common class in y_train
dummy_mf = DummyClassifier(strategy="most_frequent")
dummy_mf.fit(X_train, y_train)
y_pred_mf = dummy_mf.predict(X_val)
acc_mf = accuracy_score(y_val, y_pred_mf)
print(f"Dummy (most_frequent) accuracy: {acc_mf:.3f}")

# dummy that samples labels according to class proportion in y_train
dummy_st = DummyClassifier(strategy="stratified", random_state=42)
dummy_st.fit(X_train, y_train)
y_pred_st = dummy_st.predict(X_val)
acc_st = accuracy_score(y_val, y_pred_st)
print(f"Dummy (stratified) accuracy:   {acc_st:.3f}")


Dummy (most_frequent) accuracy: 0.545
Dummy (stratified) accuracy:   0.273


## 3. Plain Random Forest (no GA)

We now train a real model on **all 725 radiomics features** using only the 42 training cases.  
Because the training set is small and high-dimensional, we first run a 5-fold **stratified** cross-validation on the training set to get a more stable estimate.  
After that, we fit the same RF on all 42 training samples and evaluate once on the external 11-sample validation set (`X_val`, `y_val`).  
This separates “internal CV score” from “external hold-out score”.


In [40]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced_subsample"
)

# internal CV on the 42 training samples
cv_acc = cross_val_score(rf, X_train, y_train, cv=cv, scoring="accuracy")
print("RF 5-fold CV accuracy:", cv_acc)
print("RF 5-fold CV accuracy (mean):", cv_acc.mean())

if y_train.nunique() == 2:
    cv_auc = cross_val_score(rf, X_train, y_train, cv=cv, scoring="roc_auc")
    print("RF 5-fold CV AUC:", cv_auc)
    print("RF 5-fold CV AUC (mean):", cv_auc.mean())

# fit on all 42 training samples
rf.fit(X_train, y_train)

# evaluate once on the external 11-sample validation set
y_pred_val = rf.predict(X_val)
acc_val = accuracy_score(y_val, y_pred_val)
print("\nRF external accuracy:", round(acc_val, 3))

if y_val.nunique() == 2:
    y_proba_val = rf.predict_proba(X_val)[:, 1]
    auc_val = roc_auc_score(y_val, y_proba_val)
    print("RF external AUC:     ", round(auc_val, 3))

print("\nRF external classification report:")
print(classification_report(y_val, y_pred_val))


RF 5-fold CV accuracy: [0.33333333 0.44444444 0.375      0.875      0.625     ]
RF 5-fold CV accuracy (mean): 0.5305555555555556
RF 5-fold CV AUC: [0.25  0.6   0.625 0.875 0.875]
RF 5-fold CV AUC (mean): 0.645

RF external accuracy: 0.636
RF external AUC:      0.783

RF external classification report:
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.60      0.60      0.60         5

    accuracy                           0.64        11
   macro avg       0.63      0.63      0.63        11
weighted avg       0.64      0.64      0.64        11



## 4. GA-RF (paper-faithful version)

Below is a version that matches the 2022 code style much more closely.

Key points:

- we recreate their globals: `New_FS`, `y_trn`, `kfold`, `model`
- we keep their function names: `initilization_of_population`, `fitness_score`, `selection`, `crossover`, `mutation`, `generations`
- we fix only the things that would break in pandas/NumPy now:
  - `np.bool` → `bool`
  - `.iloc[train].iloc[:,chromosome]` → `.iloc[train, chromosome]`
- at the end we **actually call** `generations(...)` so you see `gen 0 ...`, `gen 1 ...` printed like their script
- this version uses **your** data: `X_train` → `New_FS`, `y_train` → `y_trn`

You can change `n_gen` or `size` later. After this, in step 5, you can take `best_chromo[-1]` and train a clean RF/XGB on that subset.


In [41]:
# ================================
# 1. set up data and base model
# ================================

# make the global variables that the original GA code expects
# New_FS: feature table to search on (your X_train)
New_FS = X_train.copy()

# y_trn: labels for GA to use during k-fold CV
y_trn = y_train.reset_index(drop=True)

# NOTE: kfold will be re-created inside the outer loop for each repeat
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# model: this is the RF the original code keeps calling inside fitness_score
# (you can change n_estimators etc. and GA will automatically use it)
model = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced_subsample"
)

# GA/global hyperparameters
size = 50                              # population size (example)
n_feat = New_FS.shape[1]               # number of features

In [42]:
# ================================
# 2. population init + fitness
# ================================
def initilization_of_population(size, n_feat):
    """
    create an initial list of chromosomes
    each chromosome is a boolean mask over features
    about 30% will be OFF (False), the rest ON (True), then shuffled
    """
    np.random.seed(42)
    INIT_OFF_RATIO = 0.3
    population = []
    for _ in range(size):
        chromosome = np.ones(n_feat, dtype=bool)
        chromosome[:int(INIT_OFF_RATIO * n_feat)] = False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    """
    for each chromosome:
      1) pick those feature columns
      2) run k-fold CV with the global RandomForest
      3) average the CV accuracy → this is the fitness
    then:
      - sort by fitness (high → low)
      - also return selection weights like the original code
    """
    scores, newtp, newfp, newtn, newfn = [], [], [], [], []

    for chromosome in population:
        tp, fp, tn, fn, acc = [], [], [], [], []

        # k-fold over the selected features
        for train_idx, test_idx in kfold.split(New_FS, y_trn):
            # pick only the features that this chromosome says "True" to
            X_train_sel = New_FS.iloc[train_idx].iloc[:, chromosome]
            X_test_sel  = New_FS.iloc[test_idx].iloc[:, chromosome]

            # train RF
            model.fit(X_train_sel, y_trn[train_idx])

            # predict
            true_labels = np.asarray(y_trn[test_idx])
            preds = model.predict(X_test_sel)

            # confusion matrix to count TP/FP/TN/FN
            tn_i, fp_i, fn_i, tp_i = confusion_matrix(true_labels, preds).ravel()

            tp.append(tp_i); fp.append(fp_i)
            tn.append(tn_i); fn.append(fn_i)

            # accuracy for this fold
            acc.append(accuracy_score(true_labels, preds))

        # average CV accuracy for this chromosome
        scores.append(np.mean(acc))

        # total counts over folds (not super important, but original code kept them)
        newtp.append(np.sum(tp)); newfp.append(np.sum(fp))
        newtn.append(np.sum(tn)); newfn.append(np.sum(fn))

    # convert to arrays for sorting
    scores = np.array(scores)
    population = np.array(population, dtype=object)

    # selection weights: higher score → higher chance to be picked
    weights = [s / scores.sum() for s in scores]

    # also sort TP/FP/TN/FN in the same order
    newtp = np.array(newtp); newfp = np.array(newfp)
    newtn = np.array(newtn); newfn = np.array(newfn)

    # sort by score descending (best first)
    inds = np.argsort(scores)[::-1]

    return (
        list(scores[inds]),
        list(population[inds]),
        list(np.array(weights)[inds]),
        list(newtp[inds]),
        list(newfp[inds]),
        list(newtn[inds]),
        list(newfn[inds]),
    )

In [43]:
# ================================
# 3. GA operators (selection / crossover / mutation)
# ================================
def selection(pop_after_fit, weights, k):
    """
    pick k chromosomes from the fitted population,
    using their fitness-based weights (roulette-wheel style)
    """
    picked = random.choices(pop_after_fit, weights=weights, k=k)
    return list(picked)


def crossover(p1, p2, crossover_rate):
    """
    single-point crossover
    two parents → two children
    sometimes we don't crossover (just copy) depending on the rate
    """
    c1, c2 = p1.copy(), p2.copy()
    if random.random() < crossover_rate and len(p1) > 2:
        pt = random.randint(1, len(p1) - 2)
        c1 = np.concatenate((p1[:pt], p2[pt:]))
        c2 = np.concatenate((p2[:pt], p1[pt:]))
    return [c1, c2]


def mutation(chromosome, mutation_rate):
    """
    go through every bit and flip it with a small probability
    this prevents GA from getting stuck too early
    """
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = not chromosome[i]


In [44]:
# ================================
# 4. full GA loop (original style)
# ================================
def generations(size, n_feat, crossover_rate, mutation_rate, n_gen):
    """
    run GA for n_gen generations in one shot
    (this is the original style)
    """
    best_chromo = []
    best_score = []

    # start with random population
    population_nextgen = initilization_of_population(size, n_feat)

    for gen in range(n_gen):
        # evaluate population
        scores, pop_after_fit, weights, tp, fp, tn, fn = fitness_score(population_nextgen)

        # best of this generation
        top_score = scores[0]
        top_chrom = pop_after_fit[0]
        print("gen", gen, "best_acc=", round(top_score, 4), "on_features=", int(top_chrom.sum()))

        # elitism (keep 2)
        elites = pop_after_fit[:2]
        k = size - 2

        # select parents
        parents = selection(pop_after_fit, weights, k)

        # make children
        children = []
        for i in range(0, len(parents), 2):
            p1 = parents[i]
            p2 = parents[(i + 1) % len(parents)]
            for child in crossover(p1, p2, crossover_rate):
                mutation(child, mutation_rate)
                children.append(child)

        # build next population
        population_nextgen = []
        for c in elites:
            population_nextgen.append(c)
        for c in children:
            if len(population_nextgen) < size:
                population_nextgen.append(c)

        # keep history
        best_chromo.append(top_chrom)
        best_score.append(top_score)

    return best_chromo, best_score


In [45]:
# ================================
# 5. single GA step (chunkable)
# ================================
def ga_one_step(population, size, crossover_rate, mutation_rate):
    """
    run GA for exactly 1 generation
    and return next_population, best_chromo, best_score
    """
    # evaluate current population
    scores, pop_after_fit, weights, tp, fp, tn, fn = fitness_score(population)

    # best in this generation
    best_score  = scores[0]
    best_chromo = pop_after_fit[0]

    # keep top-2
    elites = pop_after_fit[:2]
    k = size - 2

    # select parents for the rest
    parents = selection(pop_after_fit, weights, k)

    # crossover + mutation
    children = []
    for i in range(0, len(parents), 2):
        p1 = parents[i]
        p2 = parents[(i + 1) % len(parents)]
        for child in crossover(p1, p2, crossover_rate):
            mutation(child, mutation_rate)
            children.append(child)

    # build next population
    next_pop = []
    for e in elites:
        next_pop.append(e)
    for c in children:
        if len(next_pop) < size:
            next_pop.append(c)

    return next_pop, best_chromo, best_score


# GA BEST NPZ

In [ ]:
BEST_PATH = "../data/ga_best.npz"
TOP_K = 5

def update_top_solutions_by_run(best_path, run_id, new_score, new_mask, new_cols, top_k=5):
    """
    Keep at most `top_k` best runs.
    One entry per run_id.
    """
    if os.path.exists(best_path):
        prev = np.load(best_path, allow_pickle=True)
        prev_run_ids = prev["run_ids"].tolist()
        prev_scores  = prev["top_scores"].tolist()
        prev_masks   = prev["top_masks"].tolist()
        prev_cols    = prev["top_cols"].tolist()
    else:
        prev_run_ids, prev_scores, prev_masks, prev_cols = [], [], [], []

    # if this run_id already exists, update only if better
    if run_id in prev_run_ids:
        idx = prev_run_ids.index(run_id)
        if new_score > prev_scores[idx]:
            prev_scores[idx] = float(new_score)
            prev_masks[idx]  = new_mask
            prev_cols[idx]   = new_cols
    else:
        # new run → append
        prev_run_ids.append(run_id)
        prev_scores.append(float(new_score))
        prev_masks.append(new_mask)
        prev_cols.append(new_cols)

    # now sort by score desc and keep top_k
    order = np.argsort(prev_scores)[::-1][:top_k]

    top_run_ids = [prev_run_ids[i] for i in order]
    top_scores  = [prev_scores[i]  for i in order]
    top_masks   = [prev_masks[i]   for i in order]
    top_cols    = [prev_cols[i]    for i in order]

    np.savez(
        best_path,
        run_ids=np.array(top_run_ids, dtype=int),
        top_scores=np.array(top_scores, dtype=float),
        top_masks=np.array(top_masks, dtype=object),
        top_cols=np.array(top_cols, dtype=object),
    )
    return top_run_ids, top_scores


## Repeated GA with stratified 5-fold CV (20×) 

In [47]:
N_REPEATS = 5
GENS_PER_RUN = 3

for run in range(N_REPEATS):
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42 + run)

    pop = initilization_of_population(size=size, n_feat=n_feat)
    history_chromo, history_score = [], []

    for gen in range(GENS_PER_RUN):
        pop, bc, bs = ga_one_step(
            pop,
            size=size,
            crossover_rate=0.8,
            mutation_rate=0.05,
        )
        history_chromo.append(bc)
        history_score.append(bs)

    # best inside THIS run
    best_idx   = int(np.argmax(history_score))
    best_score = float(history_score[best_idx])
    best_mask  = history_chromo[best_idx]
    best_cols  = np.where(best_mask)[0]

    run_ids, top_scores = update_top_solutions_by_run(
        BEST_PATH,
        run_id=run,            # <-- key!
        new_score=best_score,
        new_mask=best_mask,
        new_cols=best_cols,
        top_k=5,
    )
    print("top runs:", run_ids)
    print("top scores:", top_scores)

    # optional: save last state
    np.save("../data/ga_pop.npy",     np.array(pop, dtype=object))
    np.save("../data/ga_scores.npy",  np.array(history_score, dtype=float))
    np.save("../data/ga_chromos.npy", np.array(history_chromo, dtype=object))


run 00  gen 00  best_acc=0.6500  on_features=508
run 00  gen 01  best_acc=0.6722  on_features=481
run 00  gen 02  best_acc=0.6778  on_features=457
[info] current top scores: [0.6777777777777778, 0.6777777777777778]
run 01  gen 00  best_acc=0.5944  on_features=508
run 01  gen 01  best_acc=0.5944  on_features=508
run 01  gen 02  best_acc=0.5944  on_features=508
[info] current top scores: [0.6777777777777778, 0.6777777777777778, 0.5944444444444444]
run 02  gen 00  best_acc=0.7194  on_features=508
run 02  gen 01  best_acc=0.7194  on_features=508
run 02  gen 02  best_acc=0.7194  on_features=508
[info] current top scores: [0.7194444444444444, 0.6777777777777778, 0.6777777777777778, 0.5944444444444444]
run 03  gen 00  best_acc=0.6111  on_features=508
run 03  gen 01  best_acc=0.6167  on_features=506
run 03  gen 02  best_acc=0.6167  on_features=506
[info] current top scores: [0.7194444444444444, 0.6777777777777778, 0.6777777777777778, 0.6166666666666666, 0.5944444444444444]
run 04  gen 00  best

KeyboardInterrupt: 

# Show the result 

In [ ]:
best = np.load("../data/ga_best.npz", allow_pickle=True)
run_ids    = best["run_ids"].tolist()
top_scores = best["top_scores"].tolist()
top_cols   = best["top_cols"].tolist()

print("=== TOP RUNS ===")
for rid, s, cols in zip(run_ids, top_scores, top_cols):
    print(f"run {rid}: score={s:.4f}, cols(first 5)={cols[:5]}, total={len(cols)}")


=== TOP GA SOLUTIONS ===
1. score=0.6778  cols(first 5): [1, 3, 4, 5, 6]  total=457
